In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

##Library 
import sys
# !{sys.executable} -m pip install music21 

##Utility Libraries
import os, glob, pickle, music21

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from time import time
from zipfile import ZipFile 
from music21 import *
from music21 import converter, instrument, note, chord, stream

##Keras
from keras import backend as K
if K.backend()=='tensorflow':
    K.set_image_data_format('channels_last')

import keras
from keras.models import Model, Sequential, load_model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History, TensorBoard

from keras.layers import Dense, Dropout, LSTM, Bidirectional, Activation, Input, Dense
from keras.layers import Activation, BatchNormalization, Flatten, ZeroPadding2D, Reshape
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import CuDNNGRU, CuDNNLSTM
from keras.layers.advanced_activations import LeakyReLU

from keras.models import Sequential, Model
from keras.utils import np_utils

from keras.optimizers import Adam,RMSprop,SGD
from keras.preprocessing.image import ImageDataGenerator

##Tensorflow
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

tf.debugging.set_log_device_placement(True)

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
print("Keras is installed and is version: ", tf.keras.__version__)
print("Tensorflow is installed and is version: ",  tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Keras is installed and is version:  2.2.4-tf
Tensorflow is installed and is version:  1.15.0
Num GPUs Available:  2


In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
##Add seed to get reproducible results
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [4]:
#https://www.tensorflow.org/tensorboard/get_started

"""Helper Functions"""
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            #summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()  
                
## Code adapted from Section
def plot(history):
    acc = history.history['acc']
    loss = history.history['loss']

    epochs = range(len(acc))
    plt.plot(epochs, acc, 'g-', label='Training accuracy', color = 'darkred')
    plt.xlabel("Num of Epochs")
    plt.ylabel("Accuracy")
    plt.title('Training accuracy')
    plt.legend()

    plt.figure()
 
    plt.plot(epochs, loss, 'g-', label='Training loss', color = 'darkred')
    plt.xlabel("Num of Epochs")
    plt.ylabel("Loss")
    plt.title('Training loss')
    plt.legend()

    plt.show()
    
    
# class CustomSaver(keras.callbacks.Callback):
# 	def __init__(self):
# 		self.epoch = 0	
# 	# This function is called when the training begins
# 	def on_train_begin(self, logs={}):
# 		# Initialize the lists for holding the logs, losses and accuracies
# 		self.losses = []
# 		self.acc = []
# 		self.logs = []
# 	def on_epoch_end(self, epoch, logs={}):
# 		# Append the logs, losses and accuracies to the lists
# 		self.logs.append(logs)
# 		self.losses.append(logs.get('loss'))
# 		self.acc.append(logs.get('acc')*100)
# 		# save model and plt every 50 epochs
# 		if (epoch+1) % 25 == 0:
# 			sys.stdout.write("\nAuto-saving model and plot after {} epochs to ".format(epoch+1)+"\n"+modeldirectory + modelfileprefix + "_" + str(epoch+1).zfill(3) + ".model\n"+modeldirectory + modelfileprefix + "_" + str(epoch+1).zfill(3) + ".png\n\n")
# 			sys.stdout.flush()
# 			self.model.save(modeldirectory + modelfileprefix + '_' + str(epoch+1).zfill(3) + '.model')
# 			copyfile(notesfile,modeldirectory + modelfileprefix + '_' + str(epoch+1).zfill(3) + '.notes');
# 			N = numpy.arange(0, len(self.losses))
# 			# Plot train loss, train acc, val loss and val acc against epochs passed
# 			plt.figure()
# 			plt.subplots_adjust(hspace=0.7)
# 			plt.subplot(2, 1, 1)
# 			# plot loss values
# 			plt.plot(N, self.losses, label = "train_loss")
# 			plt.title("Loss [Epoch {}]".format(epoch+1))
# 			plt.xlabel('Epoch')
# 			plt.ylabel('Loss')
# 			plt.subplot(2, 1, 2)
# 			# plot accuracy values
# 			plt.plot(N, self.acc, label = "train_acc")
# 			plt.title("Accuracy % [Epoch {}]".format(epoch+1))
# 			plt.xlabel("Epoch")
# 			plt.ylabel("Accuracy %")
# 			plt.savefig(modeldirectory + modelfileprefix + '_' + str(epoch+1).zfill(3) + '.png')
# 			plt.close()

----------
# 1.Read in Data

In [ ]:
# #Unzipping compressed data folder
# file_name = "/content/gdrive/My Drive/Pokemon MIDIs.zip"
  
# # opening the zip file in READ mode 
# with ZipFile(file_name, 'r') as zip: 
#     # printing all the contents of the zip file 
#     # zip.printdir() 
  
#     # extracting all the files 
#     print("Extracting all the files now...") 
#     zip.extractall() 
#     print("Done!") 

In [ ]:
# def get_midi_pattern():
#   for file in glob.glob("Pokemon MIDIs/*.mid"):
#     try:
#       return music21.midi.getNumbersAsList(file)
#     except:
#       print ('Error reading {}'.format(file))
#       return None

# midi_pattern = get_midi_pattern()

# def get_tones(midi_pattern):
#   """
#   returns a dict of statistics, keys: [scale_distribution,
#   """
  
#   tones = []
  
#   for track in midi_pattern:
#     for event in track:
#       if type(event) == midi.events.SetTempoEvent:
#         pass # These are currently ignored
#       elif (type(event) == midi.events.NoteOffEvent) or \
#            (type(event) == midi.events.NoteOnEvent and \
#             event.velocity == 0):
#         pass # not needed here
#       elif type(event) == midi.events.NoteOnEvent:
#         tones.append(event.data[0])
#   return tones 


-----------
# 2. Data Processing

In [5]:
def get_notes():
    notes = []
    print("Parsing...")
    for file in glob.glob("Pokemon MIDIs/*.mid"):
        midi = converter.parse(file)

      
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    
    print("Done!")
    
    return notes
    # with open('content/pianonotes', 'wb') as filepath:
    #     pickle.dump(notes, filepath)

notes = get_notes()
n_vocab = len(set(notes))

Parsing...
Done!


In [19]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, n_vocab)

-------
# 3. Model

In [21]:
K.clear_session()
def cudnn_gru(network_input, n_vocab):
    model = Sequential()
    model.add(CuDNNGRU(512,input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNGRU(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNGRU(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    opt = RMSprop(lr=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

cudnngru = cudnn_gru(network_input, n_vocab)
cudnngru.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnngru_1 (CuDNNGRU)       (None, 100, 512)          791040    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         3151872   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              4724736   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [24]:
##Add Checkpoint
filepath = "Weights/best_weights.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

callbacks_list = [checkpoint]

In [25]:
%timeit
tf.global_variables_initializer()
history_gru = cudnngru.fit(
    network_input,
    network_output,
    batch_size=50,
    epochs=60,
    verbose=1, 
    callbacks=callbacks_list)

Epoch 1/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.3896 - accuracy: 0.0135
Epoch 2/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.3485 - accuracy: 0.0132
Epoch 3/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.3393 - accuracy: 0.0136
Epoch 4/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.3096 - accuracy: 0.0135
Epoch 5/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.2900 - accuracy: 0.0139
Epoch 6/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.2811 - accuracy: 0.0140
Epoch 7/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.2720 - accuracy: 0.0144
Epoch 8/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.2621 - accuracy: 0.0137
Epoch 9/60
163585/163585 [==============================] - 290s 2ms/step - loss: 5.2549 - accuracy: 0.0139
Epoch 10/60
163585/163585 [=

In [26]:
cudnngru.save("cudnngru.mag")

In [29]:
##CuDNNGRU
prediction_output = generate_notes(cudnngru, notes, network_input, n_vocab)

create_midi(prediction_output, 'gru_midi')
# plot(history_gru)

-----------
## CuDNNGRU

In [30]:
K.clear_session()

def cudnn_lstm(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(CuDNNLSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    opt = RMSprop(lr=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

cudnnlstm = cudnn_lstm(network_input, n_vocab)
cudnnlstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6299648   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [32]:
%timeit
tf.global_variables_initializer()

history_lstm = cudnnlstm.fit(
    network_input,
    network_output,
    batch_size=64,
    epochs=30,
    verbose=1, 
    callbacks=[TrainValTensorBoard("logs/{}".format(time()), write_graph=True)])

Epoch 1/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9713 - accuracy: 0.0165
Epoch 2/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9207 - accuracy: 0.0184
Epoch 3/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9442 - accuracy: 0.0196
Epoch 4/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9589 - accuracy: 0.0187
Epoch 5/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9658 - accuracy: 0.0193
Epoch 6/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9664 - accuracy: 0.0190
Epoch 7/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9703 - accuracy: 0.0187
Epoch 8/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9719 - accuracy: 0.0184
Epoch 9/30
163585/163585 [==============================] - 287s 2ms/step - loss: 4.9736 - accuracy: 0.0186
Epoch 10/30
163585/163585 [=

In [33]:
cudnnlstm.save("cudnnlstm.mag")

--------------
# Generate new soundclips

In [28]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(set(item for item in notes))
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.ChurchBells()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

## Output

In [34]:
##CuDNNLSTM
prediction_output = generate_notes(cudnnlstm, notes, network_input, n_vocab)

create_midi(prediction_output, 'lstm_midi')
# plot(history_lstm)